###  jcmd命令使用
1. jcmd用于发送诊断命令请求到jvm, 这些请求能控制控制产生java飞行记录(java flight recording),诊断jvm和java应用;  
  jcmd需要使用和被诊断的java应用在同一台机器, 使用相同的用户名和组名

2. [jcmd command man detail page](https://docs.oracle.com/javase/8/docs/technotes/tools/windows/jcmd.html)  
3. 一个特殊命令:   
    * 打印进程的"性能表现统计值"(performance counter)
    ```shell
     jcmd <process id/main class> PerfCounter.print 
    ```
    * jcmd命令的一般格式
    ```shell
    jcmd <process id/main class> <command> [options]
    ```

4. 使用jcmd诊断命令
    * jcmd
    ```shell
    > jcmd
    5485 sun.tools.jcmd.JCmd
    2125 MyProgram
    ```
    * help
    ```
    > jcmd 3492 help
    3492:
    The following commands are available:
    JFR.stop
    JFR.start
    JFR.dump
    JFR.check
    VM.native_memory
    VM.check_commercial_features
    VM.unlock_commercial_features
    ManagementAgent.stop
    ManagementAgent.start_local
    ManagementAgent.start
    GC.rotate_log
    Thread.print
    GC.class_stats
    GC.class_histogram
    GC.heap_dump
    GC.run_finalization
    GC.run
    VM.uptime
    VM.flags
    VM.system_properties
    VM.command_line
    VM.version
    help
    > jcmd 3492 help Thread.print
    3492:
    Thread.print
    Print all threads with stacktraces.

    Impact: Medium: Depends on the number of threads.

    Permission: java.lang.management.ManagementPermission(monitor)

    Syntax : Thread.print [options]

    Options: (options must be specified using the <key> or <key>=<value> syntax)
        -l : [optional] print java.util.concurrent locks (BOOLEAN, false)
    ```
    * 打印hotspot和jvm版本
    ```shell
    > jcmd 3492 VM.version
    3492:
    Java HotSpot(TM) 64-Bit Server VM version 25.102-b14
    JDK 8.0_102
    ```
    * 打印所有系统属性 (几百行)  
    可用来查看编码问题
    ```shell
    > jcmd 3492 VM.system_properties
    3492:
    #Thu Jun 13 16:54:36 CST 2019
    java.runtime.name=Java(TM) SE Runtime Environment
    sun.boot.library.path=/usr/java/jdk1.8.0_102/jre/lib/amd64
    java.vm.version=25.102-b14
    java.vm.vendor=Oracle Corporation
    java.vendor.url=http\://java.oracle.com/
    path.separator=\:
    java.vm.name=Java HotSpot(TM) 64-Bit Server VM
    file.encoding.pkg=sun.io
    user.country=US
    sun.java.launcher=SUN_STANDARD
    sun.os.patch.level=unknown
    java.vm.specification.name=Java Virtual Machine Specification
    user.dir=/home/work/liujie32/test_jar
    java.runtime.version=1.8.0_102-b14
    ...
    file.encoding=ISO-8859-1
    ...
    ...
    ```
    * 打印出所有JVM flag
    ```shell
    > jcmd 3492 VM.flags
    3492:
    -XX:CICompilerCount=15 -XX:+FlightRecorder -XX:InitialHeapSize=2113929216 -XX:MaxHeapSize=32210157568 -XX:MaxNewSize=10736369664 -XX:MinHeapDeltaBytes=524288 -XX:NativeMemoryTracking=detail -XX:NewSize=704643072 -XX:OldSize=1409286144 -XX:+UnlockCommercialFeatures -XX:+UseCompressedClassPointers -XX:+UseCompressedOops -XX:+UseFastUnorderedTimeStamps -XX:+UseParallelGC
    ```
    * 打印类的直方图  
    包括内部引用的类和应用自身的类, 产生对象占用内存最多的类放在顶部, 后面按照内存占用大小降序排列    
    该命令同 `jmap -histo <pid>`, 但jcmd是推荐使用的 
    ```jcmd
> jcmd 3492 GC.class_histogram > class.histogram
> vim class.histogram
3492:
num     #instances         #bytes  class name
1:         13966        1055192  [C
2:          4423         492200  java.lang.Class
3:         13941         334584  java.lang.String
4:           990         313512  [B
5:             7         229488  [Lakka.dispatch.forkjoin.ForkJoinTask;
6:          5928         189696  java.util.concurrent.ConcurrentHashMap$Node
7:          2543         144800  [Ljava.lang.Object;
8:          2482          79424  java.util.HashMap$Node
9:          4311          68976  java.lang.Object
10:          1578          64904  [I
11:           448          57176  [Ljava.util.HashMap$Node;
12:            48          47584  [Ljava.util.concurrent.ConcurrentHashMap$Node;
13:           391          34408  java.lang.reflect.Method
14:           637          25480  java.util.TreeMap$Entry
15:           540          21600  java.util.LinkedHashMap$Entry
16:           413          21008  [Ljava.lang.String;
17:            99          19800  com.sun.org.apache.xerces.internal.impl.dv.xs.XSSimpleTypeDecl
```
    * heap dump文件
    heap dumo出来的堆信息文件, 可用mat工具生成堆栈信息报告
    ```jcmd
    > jcmd 3492 GC.heap_dump 2492.dump
3492:
Heap dump file created
    ```
    * 打印所有线程栈
    ```shell
    > jcmd 3492 Thread.print
    3492:
    2019-06-16 13:15:28
    Full thread dump Java HotSpot(TM) 64-Bit Server VM (25.102-b14 mixed mode):

    "VM JFR Buffer Thread" #33 daemon prio=5 os_prio=0 tid=0x00007f6088b7b800 nid=0x9730 waiting on condition [0x0000000000000000]
       java.lang.Thread.State: RUNNABLE

    "JFR request timer" #31 daemon prio=9 os_prio=0 tid=0x00007f6088a44000 nid=0x961a in Object.wait() [0x00007f6158c8d000]
       java.lang.Thread.State: WAITING (on object monitor)
        at java.lang.Object.wait(Native Method)
        at java.lang.Object.wait(Object.java:502)
        at java.util.TimerThread.mainLoop(Timer.java:526)
        - locked <0x0000000080227d48> (a java.util.TaskQueue)
        at java.util.TimerThread.run(Timer.java:505)

    "Attach Listener" #30 daemon prio=9 os_prio=0 tid=0x00007f60f0001000 nid=0x2971 waiting on condition [0x0000000000000000]
       java.lang.Thread.State: RUNNABLE

    "process reaper" #29 daemon prio=10 os_prio=0 tid=0x00007f60a0017800 nid=0x2936 waiting on condition [0x00007f61f4043000]
       java.lang.Thread.State: TIMED_WAITING (parking)
        at sun.misc.Unsafe.park(Native Method)
        - parking to wait for  <0x0000000080220690> (a java.util.concurrent.SynchronousQueue$TransferStack)
        at java.util.concurrent.locks.LockSupport.parkNanos(LockSupport.java:215)
        at java.util.concurrent.SynchronousQueue$TransferStack.awaitFulfill(SynchronousQueue.java:460)
        at java.util.concurrent.SynchronousQueue$TransferStack.transfer(SynchronousQueue.java:362)
        at java.util.concurrent.SynchronousQueue.poll(SynchronousQueue.java:941)
        at java.util.concurrent.ThreadPoolExecutor.getTask(ThreadPoolExecutor.java:1066)
        at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1127)
        at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:617)
        at java.lang.Thread.run(Thread.java:745)
    ```
    
5. 使用jcmd记录jfr
    1. 开启jfr  
    ``` jcmd 8064 VM.unlock_commercial_features ```
    2. 检查jfr是否记录完毕
    如下持续记录jvm2分钟的飞行记录
    ```bash
    > jcmd 3492 JFR.start name=myrecording delay=20s duration=2m filename=myrecording.jfr
    3492:
    Recording 2 scheduled to start in 20 s. The result will be written to:

    /home/work/liujie32/test_jar/myrecording.jfr
    
    # 查看jfr进度
    > jcmd 3492 JFR.check
    3492:
    Recording: recording=1 name="MyRecording" duration=2m filename="myrecording.jfr" compress=false (stopped)

    Recording: recording=2 name="myrecording" duration=2m filename="myrecording.jfr" compress=false (stopped)
    ```
    3. 立即创建一个jfr dump  
    这回停止相同name的jfr行为, 并将jfr立刻dump到指定文件
    ```bash
    > jcmd 3492 JFR.dump name=myrecording filename=myrecording2.jfr
    3492:
    Dumped recording "myrecording", 206.7 kB written to:

    /home/work/liujie32/test_jar/myrecording2.jfr
    ```
        